In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv("/kaggle/input/iris/Iris.csv")
df.info()

In [ ]:
df.drop(columns=["Id","Species"], inplace=True)

# algorithme de clustering DBSCAN

***le paramètre "eps=0.5" définit la distance maximale entre deux points pour qu'ils soient considérés comme appartenant au même groupe. Les points situés à une distance supérieure à eps seront considérés comme des points isolés (ou du bruit).

***le min_samples=2 spécifie le nombre minimum de points nécessaires pour former un groupe.

In [ ]:
dbscan= DBSCAN(eps=0.5, min_samples=2).fit(df)
dbscan.labels_

In [ ]:
lb=dbscan.labels_
print(set(lb))

'dbscan.labels_' renvoie les étiquettes de cluster pour chaque point dans le dataframe df. Les étiquettes sont représentées sous forme d'un tableau numpy, où chaque élément du tableau correspond à l'étiquette du cluster pour le point correspondant dans le dataframe df.

***Les étiquettes sont représentées sous forme d'entiers, avec la valeur -1 étant réservée pour les points qui sont considérés comme des points isolés (ou du bruit).

***Les clusters sont représentés par des numéros positifs: {0, 1, 2, 3, 4, 5} Soit 6 clusters.

# intégrons un score silhouette pour évaluer la qualité des clusters formés par DBSCAN.

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=2)
labels = dbscan.fit_predict(df)
silhouette_avg = silhouette_score(df, labels)
print("Le score silhouette moyen est :", silhouette_avg)


***Ici, la méthode fit_predict est utilisée pour appliquer l'algorithme DBSCAN sur les données et renvoyer les étiquettes de cluster pour chaque point. La fonction silhouette_score prend en entrée les données et les étiquettes de cluster et renvoie le score silhouette moyen pour tous les points.

# Calculons le score silhouette de chaque étiquette dans DBSCAN, on peut utiliser la fonction silhouette_samples de la bibliothèque scikit-learn. 

In [ ]:
from sklearn.metrics import silhouette_samples

dbscan = DBSCAN(eps=0.5, min_samples=2)
labels = dbscan.fit_predict(df)
silhouette_vals = silhouette_samples(df, labels)

for i in range(len(set(labels))):
    label_silhouette_vals = silhouette_vals[labels == i]
    if len(label_silhouette_vals) > 0:
        print("Le score silhouette pour l'étiquette ", i, "est :", np.mean(label_silhouette_vals))
    else:
        print("L'étiquette ", i, " n'a aucun point.")

***la fonction set est utilisée pour obtenir les étiquettes uniques dans labels. Ensuite, pour chaque étiquette unique, la fonction silhouette_samples est utilisée pour calculer le score silhouette pour chaque point qui appartient à cette étiquette. La moyenne de ces scores est ensuite calculée et affichée pour cette étiquette.


***L'étiquettes 1 a un score silhouette négatif, ce qui indique que ces points pourraient être mieux regroupés dans un autre cluster.


***Le score silhouette mesure la similarité d'un point avec les autres points de son cluster par rapport aux points des clusters voisins. Plus le score silhouette est proche de 1, mieux c'est, car cela signifie que le point est bien ajusté à son cluster et mal ajusté aux clusters voisins. Si le score est proche de -1, cela signifie que le point est mal ajusté à son cluster et bien ajusté aux clusters voisins, et si le score est proche de 0, cela signifie que le point est proche de la limite entre deux clusters.


***Dans le résultat donné, les étiquettes 0, 2, 3, 4 et 5 ont des scores silhouette positifs, ce qui est plutôt bon. L'étiquette 6 n'a aucun point, ce qui signifie qu'il n'y a pas assez de données pour calculer un score silhouette.Il s'agit de l'étiquette contenant les points isolés.